In [53]:
import math
import torch
from torch_geometric.utils import to_undirected
from torch_geometric.loader import NeighborLoader
from ogb.nodeproppred import PygNodePropPredDataset
import json
from tqdm import tqdm
import torch_geometric.utils as utils
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
from torch_geometric.datasets import Planetoid

In [64]:
K = 1
DATASET_NAME = 'PubMed'
pt_file_path = '/Users/wangzezhou/Desktop/4880作业2/Code/Data/PubMed/simteg_roberta_x.pt'

def load_dataset():
    embeddings = torch.load(pt_file_path)
    dataset = Planetoid(root='/tmp/PubMed', name=DATASET_NAME)
    data = dataset[0]
    return embeddings, data

def compute_pagerank(data):
    nx_graph = to_networkx(data, to_undirected=True)
    pagerank_values = nx.pagerank(nx_graph)
    return pagerank_values

def get_pagerank(pagerank_values, neighbours):
    values = [pagerank_values[neighbour] for neighbour in neighbours]
    return torch.tensor(values)

def compute_similarity(embeddings, node, neighbors):

    node_embedding = embeddings[node].unsqueeze(0)  # 将 node_embedding 转换为二维张量
    neighbor_embeddings = embeddings[neighbors]

    node_embedding = node_embedding / node_embedding.norm(dim=1, keepdim=True)
    neighbor_embeddings = neighbor_embeddings / neighbor_embeddings.norm(dim=1, keepdim=True)

    sim_scores = torch.mm(node_embedding, neighbor_embeddings.t()).squeeze(0)
    return sim_scores

def get_2hop_neighbors(data):
    # data.edge_index = to_undirected(data.edge_index)
    set_idx = torch.arange(data.num_nodes)  # all nodes
    loader = NeighborLoader(data, input_nodes=set_idx, num_neighbors=[-1, -1], batch_size=1, shuffle=False)

    two_hop_neighbors = {}
    for batch in tqdm(loader, desc="Processing nodes"):
        if batch.edge_index.size(1) == 0:
            continue

        node = batch.input_id.item()
        neighbors = batch.n_id.tolist()

        if node in neighbors:
            neighbors.remove(node)
        two_hop_neighbors[node] = neighbors

    return two_hop_neighbors

def find_top_k_neighbors(embeddings, two_hop_neighbors, k):
    top_k_neighbors = {}
    for node, neighbors in tqdm(two_hop_neighbors.items(), desc="Finding top-k neighbors"):
        if len(neighbors) >= k:
            neighbor_sims = compute_similarity(embeddings, node, neighbors)
            top_k = torch.topk(neighbor_sims, k=k, largest=True)
            top_k_neighbors[node] = [neighbors[i] for i in top_k.indices.tolist()]
        elif len(neighbors) > 0:
            neighbor_sims = compute_similarity(embeddings, node, neighbors)
            top_k = torch.topk(neighbor_sims, k=len(neighbors), largest=True)
            sorted_neighbors = [neighbors[i] for i in top_k.indices.tolist()]

            repeats_per_neighbor = math.ceil(k / len(sorted_neighbors))
            extended_neighbors = []
            for neighbor in sorted_neighbors:
                extended_neighbors.extend([neighbor] * repeats_per_neighbor)

            top_k_neighbors[node] = extended_neighbors[:k]
        else:
            top_k_neighbors[node] = [node] * k

    return top_k_neighbors

def prepare_for_json(data):
    if isinstance(data, dict):
        new_dict = {}
        for key, value in data.items():
            if isinstance(key, torch.Tensor):
                new_key = key.item() if key.numel() == 1 else key.tolist()
            else:
                new_key = str(key)
            new_dict[new_key] = prepare_for_json(value)
        return new_dict
    elif isinstance(data, list):
        return [prepare_for_json(element) for element in data]
    elif isinstance(data, torch.Tensor):
        return data.tolist()
    else:
        return data

def save_to_json(data, file_name):
    data = prepare_for_json(data)
    print("save begin")
    with open(file_name, 'w') as f:
        json.dump(data, f, indent=4)

def save_to_pt(data, file_name):
    torch.save(data, file_name)

def extract():
    embeddings, data = load_dataset()
    two_hop_neighbors = get_2hop_neighbors(data)
    k = K  
    top_k_neighbors = find_top_k_neighbors(embeddings, two_hop_neighbors, k)
    save_to_json(top_k_neighbors, f'./Code/Data/cosine/top_{k}_neighbors.json')

def main5():
    extract()

In [65]:
main5()

Finding top-k neighbors: 100%|██████████| 19717/19717 [00:11<00:00, 1704.27it/s]


save begin


In [76]:
K = 1
DATASET_NAME = 'PubMed'  # 'ogbn-products'  # 'ogbn-arxiv'
pt_file_path = '/Users/wangzezhou/Desktop/4880作业2/Code/Data/PubMed/simteg_roberta_x.pt'


def load_dataset(pt_file_path, set):
    embeddings = torch.load(pt_file_path)
    dataset = Planetoid(root='/tmp/PubMed', name=DATASET_NAME)
    data = dataset[0]
    
    if set == 'train':
        set_idx = data.train_mask.nonzero(as_tuple=False).view(-1)
    elif set == 'valid':
        set_idx = data.val_mask.nonzero(as_tuple=False).view(-1)
    elif set == 'test':
        set_idx = data.test_mask.nonzero(as_tuple=False).view(-1)
    else:
        raise ValueError(f"Invalid set name: {set}. Must be one of ['train', 'valid', 'test']")
    
    return embeddings, set_idx, data


def compute_similarity(embeddings, node, neighbors):

    node_embedding = embeddings[node] 
    neighbor_embeddings = embeddings[neighbors]

    node_embedding = node_embedding / node_embedding.norm(dim=1, keepdim=True)
    neighbor_embeddings = neighbor_embeddings / neighbor_embeddings.norm(dim=1, keepdim=True)

    sim_scores = torch.mm(node_embedding, neighbor_embeddings.t()).squeeze(0)
    return sim_scores


def get_2hop_neighbors(data, set_idx):
    data.edge_index = to_undirected(data.edge_index)
    loader = NeighborLoader(data, input_nodes=set_idx, num_neighbors=[-1, -1], batch_size=1, shuffle=False)

    two_hop_neighbors = {}
    for batch in tqdm(loader, desc="Processing nodes"):
        if batch.edge_index.size(1) == 0:
            continue  

        node = batch.input_id
        neighbors = batch.n_id.tolist()

        if node in neighbors:
            neighbors.remove(node)  
        two_hop_neighbors[node] = neighbors

    return two_hop_neighbors


def find_top_k_neighbors(embeddings, two_hop_neighbors, k):
    top_k_neighbors = {}
    for node, neighbors in tqdm(two_hop_neighbors.items(), desc="Finding top-k neighbors"):
        if len(neighbors) >= k:
           
            neighbor_sims = compute_similarity(embeddings, node, neighbors)
            top_k = torch.topk(neighbor_sims, k=k, largest=True)
            top_k_neighbors[node] = [neighbors[i] for i in top_k.indices.tolist()]
        elif len(neighbors) > 0:
           
            neighbor_sims = compute_similarity(embeddings, node, neighbors)
            top_k = torch.topk(neighbor_sims, k=len(neighbors), largest=True)
            sorted_neighbors = [neighbors[i] for i in top_k.indices.tolist()]

            repeats_per_neighbor = math.ceil(k / len(sorted_neighbors))
            extended_neighbors = []
            for neighbor in sorted_neighbors:
                extended_neighbors.extend([neighbor] * repeats_per_neighbor)

            top_k_neighbors[node] = extended_neighbors[:k]
        else:
   
            top_k_neighbors[node] = [node] * k
            print(top_k_neighbors[node])

    return top_k_neighbors


def prepare_for_json(data):
    if isinstance(data, dict):
        new_dict = {}
        for key, value in data.items():
            if isinstance(key, torch.Tensor):
         
                new_key = key.item() if key.numel() == 1 else key.tolist()
            else:
                new_key = str(key)
            new_dict[new_key] = prepare_for_json(value)
        return new_dict
    elif isinstance(data, list):
        return [prepare_for_json(element) for element in data]
    elif isinstance(data, torch.Tensor):
        return data.tolist()
    else:
        return data


def save_to_json(data, file_name):
    data = prepare_for_json(data)
    print("save begin")
    with open(file_name, 'w') as f:
        json.dump(data, f, indent=4)


def save_to_pt(data, file_name):
    torch.save(data, file_name)


def extract(set):
    embeddings, set_idx, data = load_dataset(pt_file_path, set)
    two_hop_neighbors = get_2hop_neighbors(data, set_idx)
    #save_to_json(two_hop_neighbors , f'./Code/Data/cora/{set}_neighbors.json')
    #save_to_pt(two_hop_neighbors, f'./Code/Data/cora/{set}_neighbors.pt')
    
    # two_hop_neighbors = torch.load(f'/Data/ogbn-products/all-roberta-large-v1/main/cached_embs/{set}_neighbors.pt')
    k = K  
    top_k_neighbors = find_top_k_neighbors(embeddings, two_hop_neighbors, k)
    save_to_json(top_k_neighbors, f'./Code/Data/cosine/{set}_top_{K}_neighbors.json')
    #save_to_pt(top_k_neighbors, f'./Code/Data/cora/{set}_top_3_neighbors.pt')


def main1():
    extract('train')
    extract('valid')
    extract('test')

In [77]:
main1()

Finding top-k neighbors: 100%|██████████| 60/60 [00:00<00:00, 1396.25it/s]


save begin


Finding top-k neighbors: 100%|██████████| 500/500 [00:00<00:00, 1797.59it/s]


save begin


Finding top-k neighbors: 100%|██████████| 1000/1000 [00:00<00:00, 1511.02it/s]

save begin
